In [3]:
import numpy as np
import pandas as pd 
churn_df=pd.read_csv("churn.csv")
col_names=churn_df.columns.tolist()

In [4]:
print("Column names:")
print(col_names)

Column names:
['State', 'Account Length', 'Area Code', 'Phone', "Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls', 'Churn?']


In [5]:
print("Sample data:")
to_show=col_names[:6]+col_names[-6:]

Sample data:


In [6]:
churn_df[to_show].head(6) 


,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,8.41,10.1,3,2.73,3,False.
5,AL,118,510,391-8027,yes,no,9.18,6.3,6,1.70,0,False.


In [7]:
churn_result=churn_df['Churn?'] 
y=np.where(churn_result=='True.',1,0)
# We don't need these columns
to_drop = ['State','Area Code','Phone','Churn?']
churn_feat_space = churn_df.drop(to_drop,axis=1)
# 'yes'/'no' has to be converted to boolean values
# NumPy converts these from boolean to 1. and 0. later
yes_no_cols = ["Int'l Plan","VMail Plan"]
churn_feat_space[yes_no_cols] = churn_feat_space[yes_no_cols] == 'yes'
# Pull out features for future use
features = churn_feat_space.columns

X = churn_feat_space.as_matrix().astype(np.float)


In [8]:
X

array([[ 128.  ,    0.  ,    1.  , ...,    3.  ,    2.7 ,    1.  ],
       [ 107.  ,    0.  ,    1.  , ...,    3.  ,    3.7 ,    1.  ],
       [ 137.  ,    0.  ,    0.  , ...,    5.  ,    3.29,    0.  ],
       ..., 
       [  28.  ,    0.  ,    0.  , ...,    6.  ,    3.81,    2.  ],
       [ 184.  ,    1.  ,    0.  , ...,   10.  ,    1.35,    2.  ],
       [  74.  ,    0.  ,    1.  , ...,    4.  ,    3.7 ,    0.  ]])

In [9]:
#This is important ! 
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=scaler.fit_transform(X) 

In [10]:
print("Unique target labels:",np.unique(y))
print(X[0])
print(len(y[y==0]))

Unique target labels: [0 1]
[ 0.67648946 -0.32758048  1.6170861   1.23488274  1.56676695  0.47664315
  1.56703625 -0.07060962 -0.05594035 -0.07042665  0.86674322 -0.46549436
  0.86602851 -0.08500823 -0.60119509 -0.0856905  -0.42793202]
2850


In [11]:
X.shape

(3333, 17)

In [12]:
from sklearn.cross_validation import KFold

def run_cv(X,y,clf_class,**kwargs):
    kf=KFold(len(y),n_folds=5,shuffle=True)
    y_pred=y.copy()
    # Iterate through folds
    for train_index,test_index in kf:
        X_train,X_test=X[train_index],X[test_index]
        y_train=y[train_index]
        # Initialize a classifier with key word arguments 
        clf=clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index]=clf.predict(X_test)
    return y_pred

In [13]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN

def accuracy(y_true,y_pred):
    return np.mean(y_true==y_pred)

print("Support vector machines:")
print("%.3f"% accuracy(y,run_cv(X,y,SVC))) 

print("Random forest:")
print("%.3f"% accuracy(y,run_cv(X,y,RF)))
print("K-nearest-neighbors:")
print("%.3f"% accuracy(y,run_cv(X,y,KNN)))

Support vector machines:
0.920
Random forest:
0.942
K-nearest-neighbors:
0.893


In [16]:
def run_prob_cv(X,y,clf_class,**kwargs):
    kf=KFold(len(y),n_folds=5,shuffle=True)
    y_prob=np.zeros((len(y),2))
    for train_index,test_index in kf:
        X_train,X_test=X[train_index],X[test_index]
        y_train=y[train_index]
        clf=clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_prob[test_index]=clf.predict_proba(X_test)
    return y_prob

In [22]:
import warnings
warnings.filterwarnings('ignore')

# Use 10 estimators so predictions are all multiples of 0.1 
pred_prob=run_prob_cv(X,y,RF,n_estimators=10)
pred_churn=pred_prob[:,1]
is_churn=y==1
counts=pd.value_counts(pred_churn) 
print(counts)

true_prob={}
for prob in counts.index:
    true_prob[prob]=np.mean(is_churn[pred_churn==prob])
    true_prob=pd.Series(true_prob)
    
# pandas-fu 
counts=pd.concat([counts,true_prob],axis=1).reset_index()
counts.columns=['pred_prob','count','true_prob'] 
counts


is_churn[pred_churn==prob]
np.mean( is_churn[pred_churn==prob])

0.0    1772
0.1     690
0.2     262
0.3     124
0.9      84
1.0      83
0.8      80
0.4      71
0.7      65
0.5      54
0.6      48
dtype: int64


0.72916666666666663

In [18]:
true_prob={}
for prob in counts.index:
    

array([ 0.1,  0. ,  0. , ...,  0.1,  0.2,  0.2])